In [225]:
import geopandas as gpd
import plotly.express as px
import json
import pandas as pd
import numpy as np

In [226]:
# Find LSOAs in multiple wards


lsoas = gpd.read_file("data/LSOAs.geojson")
wards = gpd.read_file("data/wards.geojson")

# Calculate area
lsoas["LSOA_area"] = lsoas.geometry.area


joined = gpd.sjoin(lsoas, wards, how='inner', predicate='intersects')

overlap_counts = joined.groupby(joined.index).size()
lsoas_with_multiple_wards = overlap_counts[overlap_counts > 1]
overlap_lsoas_names = lsoas.loc[lsoas_with_multiple_wards.index, "LSOA11NM"].tolist()

print(f"{len(lsoas_with_multiple_wards)} LSOAs in more than one ward")
print("LSOA names with multiple ward intersections:", overlap_lsoas_names)


4225 LSOAs in more than one ward
LSOA names with multiple ward intersections: ['City of London 001B', 'City of London 001C', 'City of London 001E', 'Barking and Dagenham 016A', 'Barking and Dagenham 015B', 'Barking and Dagenham 016B', 'Barking and Dagenham 015C', 'Barking and Dagenham 016C', 'Barking and Dagenham 015D', 'Barking and Dagenham 013A', 'Barking and Dagenham 013B', 'Barking and Dagenham 009A', 'Barking and Dagenham 009B', 'Barking and Dagenham 009C', 'Barking and Dagenham 023A', 'Barking and Dagenham 023B', 'Barking and Dagenham 008A', 'Barking and Dagenham 008C', 'Barking and Dagenham 008D', 'Barking and Dagenham 008E', 'Barking and Dagenham 001A', 'Barking and Dagenham 001B', 'Barking and Dagenham 001C', 'Barking and Dagenham 002A', 'Barking and Dagenham 002B', 'Barking and Dagenham 006A', 'Barking and Dagenham 003A', 'Barking and Dagenham 010A', 'Barking and Dagenham 010B', 'Barking and Dagenham 003B', 'Barking and Dagenham 003C', 'Barking and Dagenham 003D', 'Barking an

In [227]:
# Calculate area of LSOA within each ward it intersects


lsoas_multi = lsoas.loc[lsoas_with_multiple_wards.index]
joined_multi = joined.loc[lsoas_with_multiple_wards.index]

joined_multi = joined_multi.merge(
    wards[["geometry", "Name", "GSS_Code"]], left_on="index_right", right_index=True, suffixes=('', '_ward')
)

joined_multi["intersection_geom"] = joined_multi.apply(
    lambda row: row.geometry.intersection(row.geometry_ward), axis=1
)

# Calculate intersection area
joined_multi["intersect_area"] = joined_multi["intersection_geom"].area

# Calculate percentage
joined_multi["pct_of_lsoa_area"] = joined_multi["intersect_area"] / joined_multi["LSOA_area"] * 100

result = joined_multi[[
    "LSOA11CD", "LSOA11NM", "GSS_Code", "Name", "intersect_area", "LSOA_area", "pct_of_lsoa_area"
]].rename(columns={"Name": "Ward", "GSS_Code": "Ward_code"})

result.sort_values(by="LSOA11CD").head(10)


,LSOA11CD,LSOA11NM,Ward_code,Ward,intersect_area,LSOA_area,pct_of_lsoa_area
1,E01000002,City of London 001B,E09000001,City of London,225938.375980,226191.272990,99.888193
1,E01000002,City of London 001B,E05000367,Bunhill,252.897010,226191.272990,0.111807
2,E01000003,City of London 001C,E09000001,City of London,57187.114434,57302.966538,99.797825
2,E01000003,City of London 001C,E05000367,Bunhill,115.852104,57302.966538,0.202175
3,E01000005,City of London 001E,E05000589,Whitechapel,422.024049,190738.760504,0.221258
3,E01000005,City of London 001E,E05000585,St. Katharine's and Wapping,2500.537545,190738.760504,1.310975
3,E01000005,City of London 001E,E05000587,Spitalfields and Banglatown,1540.089984,190738.760504,0.807434
3,E01000005,City of London 001E,E09000001,City of London,186276.108927,190738.760504,97.660333
4,E01000006,Barking and Dagenham 016A,E05000031,Eastbury,88.131172,144195.846857,0.061119
4,E01000006,Barking and Dagenham 016A,E05000035,Longbridge,60.300119,144195.846857,0.041818


In [228]:
# For each LSOA, only keep row of ward it has highest percentage within


ward_counts = result.groupby('LSOA11CD')['Ward'].nunique()

# Sort
df_sorted = result.sort_values(['LSOA11CD', 'pct_of_lsoa_area'], ascending=[True, False])

df_max = df_sorted.drop_duplicates(subset='LSOA11CD', keep='first').copy()
df_max['ward_count'] = df_max['LSOA11CD'].map(ward_counts)

df_max.head(10)
# print(len(df_max))

,LSOA11CD,LSOA11NM,Ward_code,Ward,intersect_area,LSOA_area,pct_of_lsoa_area,ward_count
1,E01000002,City of London 001B,E09000001,City of London,225938.375980,226191.272990,99.888193,2
2,E01000003,City of London 001C,E09000001,City of London,57187.114434,57302.966538,99.797825,2
3,E01000005,City of London 001E,E09000001,City of London,186276.108927,190738.760504,97.660333,4
4,E01000006,Barking and Dagenham 016A,E05000026,Abbey,144047.415565,144195.846857,99.897063,3
6,E01000008,Barking and Dagenham 015B,E05000026,Abbey,192545.651669,193425.098879,99.545329,5
7,E01000009,Barking and Dagenham 016B,E05000026,Abbey,128453.553551,128591.526205,99.892705,3
8,E01000010,Barking and Dagenham 015C,E05000026,Abbey,344652.010820,348848.342174,98.797090,3
9,E01000011,Barking and Dagenham 016C,E05000026,Abbey,90146.467720,90297.679810,99.832540,3
10,E01000012,Barking and Dagenham 015D,E05000026,Abbey,137374.635727,140353.449276,97.877634,2
11,E01000013,Barking and Dagenham 013A,E05000027,Alibon,208620.701948,208625.860546,99.997527,3


In [229]:
df_max_sorted = df_max.sort_values(by='pct_of_lsoa_area', ascending=False)

In [230]:
df_max_sorted.head(10)

,LSOA11CD,LSOA11NM,Ward_code,Ward,intersect_area,LSOA_area,pct_of_lsoa_area,ward_count
4818,E01033730,Greenwich 035D,E05000222,Greenwich West,170640.866058,170640.866058,100.000000,2
4178,E01004285,Tower Hamlets 019B,E05000584,St. Dunstan's and Stepney Green,123839.530318,123839.530318,100.000000,2
2065,E01002115,Harrow 002B,E05000286,Canons,347622.251177,347622.251262,100.000000,2
3800,E01003886,Richmond upon Thames 021A,E05000530,Teddington,204505.571507,204505.571604,100.000000,2
1266,E01001286,Ealing 016E,E05000180,Hobbayne,209548.283079,209548.283223,100.000000,2
4757,E01033570,Richmond upon Thames 004G,E05000524,Kew,476052.293991,476052.294421,100.000000,2
3455,E01003532,Newham 010D,E05000481,East Ham North,111116.237132,111116.238103,99.999999,2
4520,E01004634,Wandsworth 028B,E05000628,West Hill,121110.088743,121110.089953,99.999999,2
1223,E01001242,Ealing 024C,E05000175,East Acton,206603.068600,206603.072079,99.999998,2
4710,E01033000,Hounslow 025F,E05000352,Feltham West,81840.556288,81840.558027,99.999998,2


In [231]:
df_max_sorted.tail(10)

,LSOA11CD,LSOA11NM,Ward_code,Ward,intersect_area,LSOA_area,pct_of_lsoa_area,ward_count
4750,E01033490,Islington 022H,E05000367,Bunhill,119305.063922,124850.204548,95.558565,3
2789,E01002854,Kensington and Chelsea 001C,E05000389,Golborne,125423.598642,131341.429958,95.494315,4
2550,E01002608,Hounslow 028A,E05000353,Hanworth,333561.638658,350878.258750,95.064778,2
171,E01000176,Barnet 027A,E05000049,East Finchley,209711.764164,221918.268205,94.499550,5
3709,E01003794,Redbridge 018B,E05000515,Wanstead,335110.435490,356803.702898,93.920111,4
3451,E01003528,Newham 005A,E05000481,East Ham North,79832.841469,85361.878101,93.522827,3
4813,E01033725,Hillingdon 015F,E05000340,Uxbridge North,681048.940316,916730.240266,74.291096,3
4799,E01033701,Hackney 002F,E05000231,Brownswood,164491.311717,247751.150324,66.393763,4
4708,E01032805,Southwark 022F,E05000546,Peckham,112678.319870,175293.407641,64.279839,4
4672,E01032720,Southwark 009F,E05000540,East Walworth,95130.498219,165731.441734,57.400393,4


In [232]:
# Plot LSOA and its intersecting wards


lsoa = "E01032720"


lsoas = gpd.read_file("data/LSOAs.geojson").to_crs(epsg=4326)
wards = gpd.read_file("data/wards.geojson").to_crs(epsg=4326)

lsoa_target = lsoas[lsoas["LSOA11CD"] == lsoa].copy()

intersecting_wards = gpd.sjoin(wards, lsoa_target, how='inner', predicate='intersects').copy()

# Label types
lsoa_target.loc[:, "type"] = "LSOA"
intersecting_wards.loc[:, "type"] = "Ward"

lsoa_target.loc[:, "Name"] = lsoa_target["LSOA11NM"]

# Combine both for plotting
combined = pd.concat([
    lsoa_target[["geometry", "Name", "type"]],
    intersecting_wards[["geometry", "Name", "type"]]
])

geojson_combined = json.loads(combined.to_json())

fig = px.choropleth_map(
    combined,
    geojson=geojson_combined,
    locations=combined.index,
    color="type",
    hover_name="Name",
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.5,
    height=600
)

fig.update_layout(title=f'LSOA {lsoa} and Intersecting Wards')
fig.show()

In [233]:
# Create dataframe with good overview of LSOAs within wards

single_ward_lsoa_ids = overlap_counts[overlap_counts == 1].index
single_ward_rows = joined.loc[single_ward_lsoa_ids]

single_ward_df = single_ward_rows[['LSOA11CD', 'LSOA11NM', 'GSS_Code', 'Name']].rename(columns={'Name': 'Ward', 'GSS_Code': 'Ward_code'})
multi_ward_df = df_max[['LSOA11CD', 'LSOA11NM', 'Ward_code', 'Ward']]

# Combine
final_lsoa_ward_df = pd.concat([single_ward_df, multi_ward_df], ignore_index=True)

# Sort by Ward_code
final_lsoa_ward_df = final_lsoa_ward_df.sort_values('Ward_code').reset_index(drop=True)

final_lsoa_ward_df


,LSOA11CD,LSOA11NM,Ward_code,Ward
0,E01000007,Barking and Dagenham 015A,E05000026,Abbey
1,E01000012,Barking and Dagenham 015D,E05000026,Abbey
2,E01000011,Barking and Dagenham 016C,E05000026,Abbey
3,E01000010,Barking and Dagenham 015C,E05000026,Abbey
4,E01000009,Barking and Dagenham 016B,E05000026,Abbey
...,...,...,...,...
4830,E01032739,City of London 001F,E09000001,City of London
4831,E01000005,City of London 001E,E09000001,City of London
4832,E01000002,City of London 001B,E09000001,City of London
4833,E01032740,City of London 001G,E09000001,City of London


In [234]:
# Get burglary per month per LSOA


crimes = pd.read_csv('data/all_crimes_2022-2025.csv')
crimes['Month'] = pd.to_datetime(crimes['Month'])

crimes_cleaned = crimes.dropna(subset=['Longitude', 'Latitude'])

burglary = crimes_cleaned[crimes_cleaned['Crime type'] == 'Burglary'].copy()

gdf_crimes = gpd.GeoDataFrame(
    crimes_cleaned,
    geometry=gpd.points_from_xy(crimes_cleaned['Longitude'], crimes_cleaned['Latitude']),
    crs="EPSG:4326"
)

gdf_crimes= gpd.sjoin(
    gdf_crimes,
    lsoas[['geometry', 'LSOA11NM']],
    how='left',
    predicate='within'
)

gdf_burglary = gpd.GeoDataFrame(
    burglary,
    geometry=gpd.points_from_xy(burglary['Longitude'], burglary['Latitude']),
    crs="EPSG:4326"
)

gdf_burglary = gpd.sjoin(
    gdf_burglary,
    lsoas[['geometry', 'LSOA11NM']],
    how='left',
    predicate='within'
)

monthly_burglary_counts = (
    gdf_burglary.dropna(subset=['LSOA11NM'])
    .groupby(['LSOA11NM', gdf_burglary['Month'].dt.to_period('M')])
    .size()
    .reset_index(name='Burglary_Count')
)

monthly_burglary_counts['Month'] = monthly_burglary_counts['Month'].dt.to_timestamp()

burglary_pivot = monthly_burglary_counts.pivot(index='LSOA11NM', columns='Month', values='Burglary_Count').fillna(0)

full_lsoa_months = pd.MultiIndex.from_product(
    [monthly_burglary_counts['LSOA11NM'].unique(), monthly_burglary_counts['Month'].unique()],
    names=['LSOA11NM', 'Month']
)

monthly_burglary_counts = monthly_burglary_counts.set_index(['LSOA11NM', 'Month']).reindex(full_lsoa_months, fill_value=0).reset_index()
monthly_burglary_counts

,LSOA11NM,Month,Burglary_Count
0,Barking and Dagenham 001A,2022-03-01,4
1,Barking and Dagenham 001A,2022-04-01,1
2,Barking and Dagenham 001A,2022-07-01,2
3,Barking and Dagenham 001A,2022-10-01,1
4,Barking and Dagenham 001A,2023-03-01,1
...,...,...,...
173839,Westminster 024F,2023-05-01,0
173840,Westminster 024F,2023-10-01,1
173841,Westminster 024F,2024-08-01,1
173842,Westminster 024F,2025-02-01,0


In [235]:
# Plot all LSOAs within wards and their burglary counts


selected_ward_code = "E05000026"
# YYYY-MM-DD
selected_month = "2025-02-01"

lsoas_in_ward = final_lsoa_ward_df[final_lsoa_ward_df['Ward_code'] == selected_ward_code]
burglary_for_month = monthly_burglary_counts[monthly_burglary_counts['Month'] == selected_month]

# Merge them
merged = lsoas_in_ward.merge(burglary_for_month, on="LSOA11NM", how="left")
merged["Burglary_Count"] = merged["Burglary_Count"].fillna(0)

plot_lsoas = lsoas[lsoas["LSOA11NM"].isin(merged["LSOA11NM"])]

plot_gdf = plot_lsoas.merge(merged[["LSOA11NM", "Burglary_Count"]], on="LSOA11NM")

fig = px.choropleth_map(
    plot_gdf,
    geojson=json.loads(plot_gdf.to_json()),
    locations='LSOA11NM',
    featureidkey="properties.LSOA11NM",
    color='Burglary_Count',
    color_continuous_scale="OrRd",
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.6,
    height=600
)

fig.update_layout(title=f'Burglary Count in {selected_ward_code} Ward - {selected_month}')
fig.show()


In [236]:
# Get burglary per LSOA, and calculate its share from the ward total burglary (percentage)


# YYYY-MM-DD
selected_month = "2025-02-01"

burglary_for_month = monthly_burglary_counts[
    monthly_burglary_counts["Month"] == selected_month
]

merged = final_lsoa_ward_df.merge(
    burglary_for_month,
    on="LSOA11NM",
    how="left"
)

merged["Burglary_Count"] = merged["Burglary_Count"].fillna(0)

ward_totals = merged.groupby("Ward_code")["Burglary_Count"].sum().reset_index()
ward_totals = ward_totals.rename(columns={"Burglary_Count": "Ward_Total_Burglary"})


merged = merged.merge(ward_totals, on="Ward_code", how="left")

merged["LSOA_Pct_of_Ward"] = (
    merged["Burglary_Count"] / merged["Ward_Total_Burglary"]
).fillna(0) * 100

lsoa_pct_ward = merged[[
    "Ward_code", "Ward", "LSOA11CD", "LSOA11NM",
    "Burglary_Count", "Ward_Total_Burglary", "LSOA_Pct_of_Ward"
]]

lsoa_pct_ward = lsoa_pct_ward.sort_values(
    by=["Ward_code", "Burglary_Count"], ascending=[True, False]
).reset_index(drop=True)

lsoa_pct_ward.head(10)

,Ward_code,Ward,LSOA11CD,LSOA11NM,Burglary_Count,Ward_Total_Burglary,LSOA_Pct_of_Ward
0,E05000026,Abbey,E01000010,Barking and Dagenham 015C,5.0,12.0,41.666667
1,E05000026,Abbey,E01000009,Barking and Dagenham 016B,2.0,12.0,16.666667
2,E05000026,Abbey,E01000008,Barking and Dagenham 015B,2.0,12.0,16.666667
3,E05000026,Abbey,E01000007,Barking and Dagenham 015A,1.0,12.0,8.333333
4,E05000026,Abbey,E01000012,Barking and Dagenham 015D,1.0,12.0,8.333333
5,E05000026,Abbey,E01000006,Barking and Dagenham 016A,1.0,12.0,8.333333
6,E05000026,Abbey,E01000011,Barking and Dagenham 016C,0.0,12.0,0.000000
7,E05000027,Alibon,E01000014,Barking and Dagenham 013B,2.0,3.0,66.666667
8,E05000027,Alibon,E01000013,Barking and Dagenham 013A,1.0,3.0,33.333333
9,E05000027,Alibon,E01000015,Barking and Dagenham 009A,0.0,3.0,0.000000


In [237]:
# Find the LSOA with the highest percentage for each ward


idx = lsoa_pct_ward.groupby("Ward_code")["LSOA_Pct_of_Ward"].idxmax()

top_lsoas_per_ward = lsoa_pct_ward.loc[idx].reset_index(drop=True)

# Sort by percentage
top_lsoas_per_ward = top_lsoas_per_ward.sort_values("LSOA_Pct_of_Ward", ascending=False).reset_index(drop=True)

top_lsoas_per_ward

,Ward_code,Ward,LSOA11CD,LSOA11NM,Burglary_Count,Ward_Total_Burglary,LSOA_Pct_of_Ward
0,E05000465,Lower Morden,E01003422,Merton 025E,3.0,3.0,100.0
1,E05000163,Shirley,E01001129,Croydon 026D,2.0,2.0,100.0
2,E05000078,Longlands,E01000428,Bexley 027D,1.0,1.0,100.0
3,E05000077,Lesnes Abbey,E01000416,Bexley 002A,1.0,1.0,100.0
4,E05000424,Knight's Hill,E01003087,Lambeth 031E,1.0,1.0,100.0
...,...,...,...,...,...,...,...
620,E05000080,Northumberland Heath,E01000442,Bexley 006E,0.0,0.0,0.0
621,E05000066,Blackfen and Lamorbey,E01000343,Bexley 022C,0.0,0.0,0.0
622,E05000333,Ickenham,E01002462,Hillingdon 011D,0.0,0.0,0.0
623,E05000318,Rainham and Wennington,E01002342,Havering 030A,0.0,0.0,0.0


In [238]:
print("Number of wards with 100% of burglaries in 1 LSOA:", (top_lsoas_per_ward["LSOA_Pct_of_Ward"] == 100).sum())
print("Number of wards with no burglaries:", (top_lsoas_per_ward["LSOA_Pct_of_Ward"] == 0).sum())

Number of wards with 100% of burglaries in 1 LSOA: 65
Number of wards with no burglaries: 18


In [239]:
# Add Police_officers column by rounding down percentage


lsoa_pct_ward["Police_officers"] = np.floor(lsoa_pct_ward["LSOA_Pct_of_Ward"]).astype(int)

lsoa_pct_ward.head(10)

,Ward_code,Ward,LSOA11CD,LSOA11NM,Burglary_Count,Ward_Total_Burglary,LSOA_Pct_of_Ward,Police_officers
0,E05000026,Abbey,E01000010,Barking and Dagenham 015C,5.0,12.0,41.666667,41
1,E05000026,Abbey,E01000009,Barking and Dagenham 016B,2.0,12.0,16.666667,16
2,E05000026,Abbey,E01000008,Barking and Dagenham 015B,2.0,12.0,16.666667,16
3,E05000026,Abbey,E01000007,Barking and Dagenham 015A,1.0,12.0,8.333333,8
4,E05000026,Abbey,E01000012,Barking and Dagenham 015D,1.0,12.0,8.333333,8
5,E05000026,Abbey,E01000006,Barking and Dagenham 016A,1.0,12.0,8.333333,8
6,E05000026,Abbey,E01000011,Barking and Dagenham 016C,0.0,12.0,0.000000,0
7,E05000027,Alibon,E01000014,Barking and Dagenham 013B,2.0,3.0,66.666667,66
8,E05000027,Alibon,E01000013,Barking and Dagenham 013A,1.0,3.0,33.333333,33
9,E05000027,Alibon,E01000015,Barking and Dagenham 009A,0.0,3.0,0.000000,0
